# IMDB 영화평 감성분석 (이진 분류)

In [1]:
import numpy as np
import pandas as pd 

In [4]:
df = pd.read_csv('data/labeledTrainData.tsv', sep='\t', quoting=3)
df.head(3)

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         25000 non-null  object
 1   sentiment  25000 non-null  int64 
 2   review     25000 non-null  object
dtypes: int64(1), object(2)
memory usage: 586.1+ KB


In [6]:
df.review[0][:1000]

'"With all this stuff going down at the moment with MJ i\'ve started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ\'s feeling towards the press and also the obvious message of drugs are bad m\'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally

- 전처리

In [7]:
# <br /> 태그는 공백으로 변환
df.review = df.review.str.replace('<br />',' ')

In [8]:
# 구둣점, 숫자 제거 --> 영어 이외의 문자는 공백으로 변환
df.review = df.review.str.replace('[^A-Za-z]',' ')

<ipython-input-8-dd96667435e9>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df.review = df.review.str.replace('[^A-Za-z]',' ')


In [9]:
df.review[0][:1000]

' With all this stuff going down at the moment with MJ i ve started listening to his music  watching the odd documentary here and there  watched The Wiz and watched Moonwalker again  Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent  Moonwalker is part biography  part feature film which i remember going to see at the cinema when it was originally released  Some of it has subtle messages about MJ s feeling towards the press and also the obvious message of drugs are bad m kay   Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring  Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him   The actual feature film bit when it finally starts is only on for 

- Train/Test dataset 분리

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    df.review, df.sentiment, stratify=df.sentiment, random_state=2021
)
y_train.value_counts()

0    9375
1    9375
Name: sentiment, dtype: int64

In [11]:
y_test.value_counts()

0    3125
1    3125
Name: sentiment, dtype: int64

- CountVectorizer로 변환

In [14]:
from sklearn.feature_extraction.text import CountVectorizer
cvect = CountVectorizer(stop_words='english', ngram_range=(1,2))

In [13]:
# 이런 방법으로 하면 안됨
cvect.fit_transform(X_train).shape, cvect.fit_transform(X_test).shape

((18750, 1384106), (6250, 540143))

In [16]:
# 이와 같은 방법을 사용해야 함
cvect.fit(X_train)
X_train_cv = cvect.transform(X_train)
X_test_cv = cvect.transform(X_test)
X_train_cv.shape, X_test_cv.shape

((18750, 1384106), (6250, 1384106))

- LogisticRegression

In [19]:
from sklearn.linear_model import LogisticRegression
lrc = LogisticRegression(max_iter=300)
%time lrc.fit(X_train_cv, y_train)

Wall time: 28 s


LogisticRegression(max_iter=300)

In [20]:
lrc.score(X_test_cv, y_test)

0.8864

- TfidfVectorizer로 변환/학습/평가

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer
tvect = TfidfVectorizer(stop_words='english', ngram_range=(1,2))
tvect.fit(X_train)
X_train_tv = tvect.transform(X_train)
X_test_tv = tvect.transform(X_test)
X_train_tv.shape, X_test_tv.shape

((18750, 1384106), (6250, 1384106))

In [22]:
from sklearn.linear_model import LogisticRegression
lrc = LogisticRegression(max_iter=300)
%time lrc.fit(X_train_tv, y_train)
lrc.score(X_test_tv, y_test)

Wall time: 12.7 s


0.87904

- 모델 저장하고 불러오기

In [23]:
import joblib

In [25]:
joblib.dump(tvect, 'model/imdb_tvect.pkl')
joblib.dump(lrc, 'model/imdb_lrc.pkl')

['model/imdb_lrc.pkl']

In [26]:
new_tvect = joblib.load('model/imdb_tvect.pkl')
new_lrc = joblib.load('model/imdb_lrc.pkl')

In [27]:
new_X_test_tv = new_tvect.transform(X_test)
new_lrc.score(new_X_test_tv, y_test)

0.87904